In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('datasets/train_reviews.csv')
data.head()

,Review,Sentiment
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [3]:
data = data.dropna(axis = 0, how ='any')
len(data)

24641

In [4]:
data = data.drop(['Sentiment'],axis=1)
data.rename(columns = {'Review': 'Text'}, inplace = True)

In [5]:
category = {}

category['feedback'] = ['opinion', 'think', 'idea', 'suggest', 'recommend', 'propose', 'improve', 'add', 'advise', 'propose', 'comment', 
                        'commend', 'tip', 'view', 'mind', 'conclusion', 'feel', 'feedback', 'feel', 'usually']

In [6]:
for k, v in category.items():
    v = {w.lower() for w in v}
    category[k] = v

In [7]:
# category['feedback']

In [8]:
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
import string
import spacy
import re

punctuation_auto = list(string.punctuation)
punctuation_custom = [ '...', '…' ]
punctuation = np.unique(punctuation_auto+punctuation_custom)

def trends_tags(data,str_input):
    tag_re = '[^ ]*#[^ ]*'
    tags_list = []
    temp = []
    for index, row in data.iterrows():
        tokens = str(row[str_input]).split() 
        for word in tokens:
            if (re.match(tag_re, word)):
                tags_list.append(word)

    common_words = Counter(tags_list).most_common(3)
    for key, value in common_words:
        temp.append({'name': key, 'value': value})

    return temp

def light_text_cleaning_english(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)

    # remove numbers
    text = re.sub(r'\d+', '', text)
    # remove links
    text = re.sub('http[s]?://\S+', '', text)
    # remove word with tweethandle @name
    text = re.sub('[^ ]*@[^ ]*', '', text)

    return text

# def text_cleaning_english(text,mynlp):

#     # remove numbers
#     text = re.sub(r'\d+', '', text)
#     # remove links
#     text = re.sub('http[s]?://\S+', '', text)
#     # remove word with tweethandle @name
#     text = re.sub('[^ ]*@[^ ]*', '', text)

#     doc = mynlp(text)
    
#     # Lemmatization
#     tokens = []
#     for token in doc:
#         if token.lemma_ != "-PRON-":
#             temp = token.lemma_.lower().strip()
#         else:
#             temp = token.lower_
#         tokens.append(temp)
    
#     # Remove punctuation and stopwords
#     cleaned_tokens = []
#     for token in tokens:
#         if token not in list(STOP_WORDS) and token not in punctuation:
#             cleaned_tokens.append(token)

#     # traverse in the string     
#     complete_sentence = ' '.join([str(words) for words in cleaned_tokens])

#     return complete_sentence

def most_used_words(data,str_input):

    words_list = []
    temp = []
    for index, row in data.iterrows():
        # print(type(row['cleaned_'+str_input]))
        tokens = str(row['Cleaned_'+str_input]).split() 
        for word in tokens:
            if word not in list(STOP_WORDS):
                words_list.append(word)

    common_words = Counter(words_list).most_common(50)
    for key, value in common_words:
        temp.append({'name': key, 'value': value})

    return temp

def word_finder(data,str_input):
    words_list = []
    predicted = []
    for index, row in data.iterrows():
        # print(row['Cleaned_'+str_input])
        check = False
        tokens = str(row['Cleaned_'+str_input]).split() 
        for word in tokens:
            if word in list(category['feedback']):
                check = True
        
        if check:
            words_list.append(str(row['Cleaned_'+str_input]))
            predicted.append('Feedback')
        else:
            predicted.append('Non-Feedback')

    return words_list, predicted

In [ ]:
test = {'Text': [
    'In my opinion this is a good product #goodquality',
    'I think it would be great if you could add more features #highquality',
    'I like it #goodquality #mychoice',
    'Good product and quality',
    'This product is recommended for all',
    'Would buy the product again next time #goodquality #recommended',
]}
test_data = pd.DataFrame(test)

In [ ]:
# get text contain the words
tag_list = trends_tags(test_data,'Text')
tag_list

[{'name': '#goodquality', 'value': 3},
 {'name': '#highquality', 'value': 1},
 {'name': '#mychoice', 'value': 1}]

In [ ]:
text_cleaning = lambda x: light_text_cleaning_english(x)
data['Cleaned_Text'] = pd.DataFrame(data['Text'].apply(text_cleaning))

In [ ]:
# get text contain the words
word_list, data_predicted = word_finder(data,'Text')

In [ ]:
data['Feedback_Label'] = data_predicted

In [ ]:
data.head()

In [ ]:
new = {'Text': word_list }
new_data = pd.DataFrame(new)

In [ ]:
new_data.head()

,Text
0,i love tracy reese dresses but this one is not...
1,i love this dress i usually get an xs but it r...
2,this is a nice choice for holiday gatherings i...
3,material and color is nice the leg opening is...
4,i love the look and feel of this tulle dress i...


In [ ]:
len(new_data)

6450

In [ ]:
# new_data.to_csv('opinion_data.csv', index=False)

In [ ]:
# text_cleaning = lambda x: light_text_cleaning_english(x)
# new_data['Cleaned_Text'] = pd.DataFrame(new_data['Text'].apply(text_cleaning))

In [ ]:
# common_list = most_used_words(new_data,'Text')
# common_list